In [1]:
import pickle
import shutil
import csv
import os
import librosa
import librosa.display
import pyaudio
import wave
import keras
import pyaudio  
import sys
import numpy as np
import matplotlib.pyplot as plt
import speech_recognition as speech_r
import scipy.signal as signal
import random as rn
import tensorflow as tf
import pandas as pd
from pydub import AudioSegment
from time import sleep
from matplotlib import cm
from python_speech_features import logfbank, fbank
from scipy.fftpack import dct
from silence_remover.silence_detector import SilenceDetector
from silence_remover.filter_generator import FilterGenerator
from silence_remover.media_editor import MediaEditor
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
from sklearn.linear_model import LogisticRegression
from keras.layers import Dense
from keras import Input
from keras.engine import Model
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.layers import Dense,concatenate, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, MaxPooling1D,Multiply,Conv1D, Conv2D, MaxPooling2D
from playsound import playsound
from sklearn.model_selection import train_test_split

C:\Users\thakd\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\thakd\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\thakd\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\thakd\anaconda3\envs\test\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
save_path = "./train_data/"
user_id = os.listdir(save_path) # 경로에 있는 폴더 리스트
user_id.sort() # 폴더 오름차순으로 정렬

r = speech_r.Recognizer()  

print("User List : ")
testInt = 0

## 경로에 있는 사용자 파일 목록 출력 시작 ##

for i, item in zip(range(len(user_id)), user_id) :
    print('[{:d}] '.format(i) + item)

    file_list = os.listdir(save_path + str(user_id[i] + "/"))
    file_list.sort()

    if os.path.exists("./train_mfcc/" + str(user_id[i] + "/")) :
        shutil.rmtree("./train_mfcc/" + str(user_id[i] + "/"))
        
    for j, item in zip(range(len(file_list)), file_list) :
        testInt = testInt + 1
        file_name = save_path + str(user_id[i]) + "/" + str(file_list[j])
        print(file_name)
        
        if not file_name.endswith(".wav") :
            
            (path, file_extension) = os.path.splitext(file_name)
            file_extension = file_extension.replace('.', '')

            track = AudioSegment.from_file(file_name, file_extension)
            wav_file_name = file_name.replace(file_extension, 'wav')

            wav_path = wav_file_name
            print('CONVERTING : ' + str(file_name))
            file_handle = track.export(wav_path, format = 'wav')
            
            os.remove(file_name)
        else :
            wav_path = file_name
            
## 경로에 있는 사용자 파일 목록 출력 끝 ##

###### 이 부분은 인증을 위한 음성을 녹음한 파일에 대한 전처리과정 ######

        hop_length = 256
        n_mels = 40
        n_fft = 512
        sr = 16000 ## STFT 를 위한 값 설정
            
        temp_file = wav_path.replace(".wav", "t.wav")
        fw = open(temp_file, "wb")
        fw.write(audio.get_wav_data())
        fw.close()
            
        y, sr = librosa.load(temp_file, sr = 16000, mono = True, duration = 10)
        y = y[0 : int(5.0 * sr)]
            
        stft = librosa.stft(y = y, n_fft = n_fft, hop_length = hop_length, win_length = None, window = 'hann',
            center = False, dtype = np.complex64) ## STFT 실행
            
        abs_STFT = np.abs(stft)
    
        mel = librosa.filters.mel(sr = sr, n_fft = n_fft, n_mels = n_mels)
        mel /= np.max(mel, axis = -1)[:, None]
        mel_S = np.dot(mel, abs_STFT) ## mel 변환

        mel_spec_test = 20 * np.log10(np.maximum(mel_S, 1e-5)) 
    
        num_ceps = 19
        cep_lifter = 22
    
        mfcc = dct(mel_spec_test.T, type=2 ,axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] ## Mfcc 변환
        (nframes, ncoeff) = mfcc.shape

        n = np.arange(ncoeff)
    
        # To de-emphasize higher MFCCs (for Noise)
        lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter) 
        mfcc_lifted = mfcc * lift ## Lifter 함수 적용
    
        feature_save_path = "./train_mfcc/" + str(user_id[i] + "/")

        if not os.path.exists(feature_save_path) :
            os.makedirs(feature_save_path)
            feature_result_name = str(user_id[i]) + "_" + str(0)
                
        else :
            feature_files = os.listdir(feature_save_path)
            feature_files = [file for file in feature_files if file.endswith(".npy")]
            feature_result_name = str(user_id[i]) + "_" + str(len(feature_files))
        
        if os.path.exists(feature_save_path + feature_result_name + ".npy") :
            os.remove(feature_save_path + feature_result_name + ".npy")
           
        print(feature_result_name + "' File Write Finish !")
        np.save(feature_save_path + feature_result_name, mfcc_lifted) # x_save.npy
        print(" Saved File Name : " + feature_result_name)

        os.remove(temp_file)
        sleep(1)
            
print(testInt) ## 변환된 파일 개수

User List :
[0] 강은기
./train_data/강은기/강은기01.wav
강은기_0' File Write Finish !
 Saved File Name : 강은기_0
./train_data/강은기/강은기02.wav
강은기_1' File Write Finish !
 Saved File Name : 강은기_1
./train_data/강은기/강은기03.wav
강은기_2' File Write Finish !
 Saved File Name : 강은기_2
./train_data/강은기/강은기04.wav
강은기_3' File Write Finish !
 Saved File Name : 강은기_3
./train_data/강은기/강은기05.wav
강은기_4' File Write Finish !
 Saved File Name : 강은기_4
./train_data/강은기/강은기06.wav
강은기_5' File Write Finish !
 Saved File Name : 강은기_5
./train_data/강은기/강은기07.wav
강은기_6' File Write Finish !
 Saved File Name : 강은기_6
./train_data/강은기/강은기08.wav
강은기_7' File Write Finish !
 Saved File Name : 강은기_7
./train_data/강은기/강은기09.wav
강은기_8' File Write Finish !
 Saved File Name : 강은기_8
./train_data/강은기/강은기10.wav
강은기_9' File Write Finish !
 Saved File Name : 강은기_9
./train_data/강은기/강은기11.wav
강은기_10' File Write Finish !
 Saved File Name : 강은기_10
./train_data/강은기/강은기12.wav
강은기_11' File Write Finish !
 Saved File Name : 강은기_11
./train_data/강은기/강은기13.wav
강

./train_data/윤동준/윤동준03.wav
윤동준_2' File Write Finish !
 Saved File Name : 윤동준_2
./train_data/윤동준/윤동준04.wav
윤동준_3' File Write Finish !
 Saved File Name : 윤동준_3
./train_data/윤동준/윤동준05.wav
윤동준_4' File Write Finish !
 Saved File Name : 윤동준_4
./train_data/윤동준/윤동준06.wav
윤동준_5' File Write Finish !
 Saved File Name : 윤동준_5
./train_data/윤동준/윤동준07.wav
윤동준_6' File Write Finish !
 Saved File Name : 윤동준_6
./train_data/윤동준/윤동준08.wav
윤동준_7' File Write Finish !
 Saved File Name : 윤동준_7
./train_data/윤동준/윤동준09.wav
윤동준_8' File Write Finish !
 Saved File Name : 윤동준_8
./train_data/윤동준/윤동준10.wav
윤동준_9' File Write Finish !
 Saved File Name : 윤동준_9
./train_data/윤동준/윤동준11.wav
윤동준_10' File Write Finish !
 Saved File Name : 윤동준_10
./train_data/윤동준/윤동준12.wav
윤동준_11' File Write Finish !
 Saved File Name : 윤동준_11
./train_data/윤동준/윤동준13.wav
윤동준_12' File Write Finish !
 Saved File Name : 윤동준_12
./train_data/윤동준/윤동준14.wav
윤동준_13' File Write Finish !
 Saved File Name : 윤동준_13
./train_data/윤동준/윤동준15.wav
윤동준_14' File Writ

./train_data/주홍식/주홍식05.wav
주홍식_4' File Write Finish !
 Saved File Name : 주홍식_4
./train_data/주홍식/주홍식06.wav
주홍식_5' File Write Finish !
 Saved File Name : 주홍식_5
./train_data/주홍식/주홍식07.wav
주홍식_6' File Write Finish !
 Saved File Name : 주홍식_6
./train_data/주홍식/주홍식08.wav
주홍식_7' File Write Finish !
 Saved File Name : 주홍식_7
./train_data/주홍식/주홍식09.wav
주홍식_8' File Write Finish !
 Saved File Name : 주홍식_8
./train_data/주홍식/주홍식10.wav
주홍식_9' File Write Finish !
 Saved File Name : 주홍식_9
./train_data/주홍식/주홍식11.wav
주홍식_10' File Write Finish !
 Saved File Name : 주홍식_10
./train_data/주홍식/주홍식12.wav
주홍식_11' File Write Finish !
 Saved File Name : 주홍식_11
./train_data/주홍식/주홍식13.wav
주홍식_12' File Write Finish !
 Saved File Name : 주홍식_12
./train_data/주홍식/주홍식14.wav
주홍식_13' File Write Finish !
 Saved File Name : 주홍식_13
./train_data/주홍식/주홍식15.wav
주홍식_14' File Write Finish !
 Saved File Name : 주홍식_14
./train_data/주홍식/주홍식16.wav
주홍식_15' File Write Finish !
 Saved File Name : 주홍식_15
./train_data/주홍식/주홍식17.wav
주홍식_16' File 

In [4]:
save_path = "./test_data/"
user_id = os.listdir(save_path)
user_id.sort()

r = speech_r.Recognizer() 

print("User List :")
testInt = 0

## 경로에 있는 사용자 파일 목록 출력 시작 ##

for i, item in zip(range(len(user_id)), user_id) :
    print('[{:d}] '.format(i) + item)

    file_list = os.listdir(save_path + str(user_id[i] + "/"))
    file_list.sort()

    
    if os.path.exists("./test_mfcc/" + str(user_id[i] + "/")) :
        shutil.rmtree("./test_mfcc/" + str(user_id[i] + "/"))
        
    for j, item in zip(range(len(file_list)), file_list) :
        testInt = testInt + 1
        file_name = save_path + str(user_id[i]) + "/" + str(file_list[j])
        print(file_name)
        
        if not file_name.endswith(".wav") :
            
            (path, file_extension) = os.path.splitext(file_name)
            file_extension = file_extension.replace('.', '')

            track = AudioSegment.from_file(file_name, file_extension)
            wav_filename = file_name.replace(file_extension, 'wav')

            wav_path = wav_filename
            print('CONVERTING: ' + str(file_name))
            file_handle = track.export(wav_path, format = 'wav')
            os.remove(file_name)
        else :
            wav_path = file_name

            
## 경로에 있는 사용자 파일 목록 출력 끝 ##

###### 이 부분은 인증을 위한 음성을 녹음할 파일에 대한 전처리과정 ######            

        hop_length = 256
        n_mels = 40
        n_fft = 512
        sr = 16000

        temp_file = wav_path.replace(".wav", "Temp.wav")
        fw = open(temp_file, "wb")
        fw.write(audio.get_wav_data())
        fw.close()
            
        y, sr = librosa.load(temp_file, sr = 16000, mono = True, duration = 10)
        y = y[0 : int(5.0 * sr)]
            
        stft = librosa.stft(y = y, n_fft = n_fft, hop_length = hop_length, win_length = None, window = 'hann',
             center = False, dtype = np.complex64)
        abs_STFT = np.abs(stft)
    
        mel = librosa.filters.mel(sr = sr, n_fft = n_fft, n_mels = n_mels)
        mel /= np.max(mel, axis = -1)[:, None]
        mel_S = np.dot(mel, abs_STFT)
        
        mel_spec_test = 20 * np.log10(np.maximum(mel_S, 1e-5))
    
        num_ceps = 19
        cep_lifter = 22
        
        mfcc = dct(mel_spec_test.T, type = 2 ,axis = 1, norm = 'ortho')[:, 1 : (num_ceps + 1)] 
        (nframes, ncoeff) = mfcc.shape

        n = np.arange(ncoeff)
    
        # To de-emphasize higher MFCCs (for Noise)
        lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)
        mfcc_lifted = mfcc * lift
    
        feature_save_path = "./test_mfcc/" + str(user_id[i] + "/")


        if not os.path.exists(feature_save_path):
            os.makedirs(feature_save_path)
            feature_result_name = str(user_id[i]) + "_" + str(0)
                
        else :
            feature_files = os.listdir(feature_save_path)
            feature_files = [file for file in feature_files if file.endswith(".npy")]
            feature_result_name = str(user_id[i]) + "_" + str(len(feature_files))
        
        if os.path.exists(feature_save_path + feature_result_name + ".npy"):
            os.remove(feature_save_path + feature_result_name+".npy")
            
        print(feature_result_name + "' File Write Finish !")
        np.save(feature_save_path + feature_result_name, mfcc_lifted) # x_save.npy
        print(" Saved File Name : "+ feature_result_name)

        os.remove(temp_file)
        sleep(1)
        
print(testInt) ## 변환된 파일 개수

User List :
[0] 강은기
./test_data/강은기/test_강은기.wav
강은기_0' File Write Finish !
 Saved File Name : 강은기_0
./test_data/강은기/test_강은기_1.wav
강은기_1' File Write Finish !
 Saved File Name : 강은기_1
./test_data/강은기/test_강은기_2.wav
강은기_2' File Write Finish !
 Saved File Name : 강은기_2
./test_data/강은기/test_강은기_3.wav
강은기_3' File Write Finish !
 Saved File Name : 강은기_3
./test_data/강은기/test_강은기_4.wav
강은기_4' File Write Finish !
 Saved File Name : 강은기_4
./test_data/강은기/test_강은기_5.wav
강은기_5' File Write Finish !
 Saved File Name : 강은기_5
[1] 강은서
./test_data/강은서/test_강은서.wav
강은서_0' File Write Finish !
 Saved File Name : 강은서_0
./test_data/강은서/test_강은서_1.wav
강은서_1' File Write Finish !
 Saved File Name : 강은서_1
./test_data/강은서/test_강은서_2.wav
강은서_2' File Write Finish !
 Saved File Name : 강은서_2
./test_data/강은서/test_강은서_3.wav
강은서_3' File Write Finish !
 Saved File Name : 강은서_3
./test_data/강은서/test_강은서_4.wav
강은서_4' File Write Finish !
 Saved File Name : 강은서_4
./test_data/강은서/test_강은서_5.wav
강은서_5' File Write Finish !
 Save

# 